In [9]:
%load_ext pycodestyle_magic

In [ ]:
### This cell is uneditable ###
import json
import tempfile

result = []
# Build a network with
#  convolution layer 1 having "filters1" filters
#  convolution layer 2 having "filters2" filters
#  convolution layer 3 having "filters3" filters or None to remove
#  fully connected layer of neurons with "layer4" neurons
#  fully connected layer of neurons with "layer5" neurons
#
# PICK ONE
#LAYER_CONFIG = "large"
#         "filters1": 50
#         "filters2": 75
#         "filters3": 100
#         "layer4": 500
#         "layer5": 250
#LAYER_CONFIG = "medium"
#         "filters1": 25
#         "filters2": 50
#         "filters3": 75
#         "layer4": 250
#         "layer5": 100
#LAYER_CONFIG = "small"
#         "filters1": 25
#         "filters2": 25
#         "filters3": 25
#         "layer4": 125
#         "layer5": 50
#LAYER_CONFIG = "tiny"
#         "filters1": 25
#         "filters2": 25
#         "filters3": None
#         "layer4": 75
#         "layer5": 25

labels = ["plane", "car", "tank"]
result.append(json.dumps([1500, 300]))
#JSON([1500, 300])

In [52]:
# GOAL: You are configuring a CNN to perform image classification.
#       There are three classes: Planes, Tanks, and Automobiles drawn from the CIFAR-10
#       and CIFAR-100 datasets collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.
#       Training data: 500 examples of each class. Test data: 100 examples of each class.
#       You need to determine which settings help improve the classification and why?
#-----------------------------------------------------
# LAYER_CONFIG: What configuration is best for this example?
#       Look above in the "Setup" section to see what layer configurations are available
#       for training and testing. The network can have up to 3 convolutional layers
#       and 2 fully-connected layers.
#-----------------------------------------------------
# possible values: tiny, small, medium or large
LAYER_CONFIG = "medium"
# TRAINING_EPOCHS: How many epochs should the network train?
#-----------------------------------------------------
# possible values: 30 or 60
TRAINING_EPOCHS = 60

In [53]:
### This cell is uneditable ###
if TRAINING_EPOCHS != 30 and TRAINING_EPOCHS != 60:
    raise Exception("TRAINING_EPOCHS must be 30 or 60")
if LAYER_CONFIG not in ["tiny", "small", "medium", "large"]:
    raise Exception("invalid value for LAYER_CONFIG")

dir = tempfile.gettempdir()
prefix = dir + "/Epoch" + str(TRAINING_EPOCHS) + "_" + LAYER_CONFIG + "_"
# LOAD ACCURACY DATA and PREDICTIONS
with open(prefix + "accuracy.json") as IN:
    plot_dict = json.load(IN)

with open(prefix + "predictions.json") as IN:
    predictions_dict = json.load(IN)

    y_pred_test = predictions_dict["y_pred_test"]
    y_proba = predictions_dict["y_proba"]

import random
import numpy as np
experiment = []
# 0: airplane, 1: automobile, 2: tank.
choices = [0, 1, 2]
cat0 = 0
cat1 = 100
cat2 = 200
for i in range(len(y_pred_test)):
        c = random.choice(choices)
        if not(choices):
            raise Exception("ERROR: generating test data")
        if c == 0:
            filename = "airplane_" + str(cat0) + ".png"
            y_true = 0
            pred = int(y_pred_test[cat0])
            prob = float(y_proba[cat0])
            cat0 += 1
            if cat0 >= 100:
                choices.remove(0)
        elif c == 1:
            filename = "automobile_" + str(cat1) + ".png"
            y_true = 1
            pred = int(y_pred_test[cat1])
            prob = float(y_proba[cat1])
            cat1 += 1
            if cat1 >= 200:
                choices.remove(1)
        else:
            filename = "tank_" + str(cat2) + ".png"
            y_true = 2
            pred = int(y_pred_test[cat2])
            prob = float(y_proba[cat2])
            cat2 += 1
            if cat2 >= 300:
                choices.remove(2)
        experiment.append({
            "inputImage": filename,
            "realLabel": labels[y_true],
            "classifierLabel": labels[pred],
            "confidence": prob})
output = np.array_split(experiment,5)

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

result.append(json.dumps(output, cls= NumpyEncoder))